In [10]:
import pandas as pd
import numpy as np
import pickle

# Definimos path
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

pondera = 'PONDERA_new'
var = 'PROVINCIA'
df_temp = df_people_pool.query('CH06 >= 14')

In [107]:
# Diccionario regiones ENUT

dict_regiones_ENUT = {1: 'Gran Buenos Aires',
                    2: 'Pampeana',
                    3: 'Noroeste',
                    4: 'Noreste',
                    5: 'Cuyo',
                    6: 'Patagonia'}

# Cargo ENUT
pathfolder_ENUT =  pathdata + 'data_input/ENUT_2021/Bases de datos/'
filename = 'enut2021_base.txt'
df_enut = pd.read_csv(pathfolder_ENUT + filename, sep='|')
columnas = pd.DataFrame(df_enut.columns)

## Indicadores

In [11]:
## Empleo publico

df_ocupados = df_temp[df_temp['ESTADO']==1][[var, pondera, 'CH04']]
df_ocupados_publico = df_temp[(df_temp['ESTADO']==1) & (df_temp['PP04A']==1)][[var, pondera, 'CH04']]
df_tasa = df_ocupados_publico.groupby(['CH04',var])[pondera].sum() / df_ocupados.groupby(['CH04',var])[pondera].sum()
df_empleo_publico = df_tasa.unstack(level=0)*100
df_empleo_publico = df_empleo_publico.rename(columns={1:'Varon', 2:'Mujer'})
df_empleo_publico['Ratio'] = df_empleo_publico['Mujer']/df_empleo_publico['Varon']*100
df_empleo_publico.index = df_empleo_publico.index.map(dict_cod_provincia)

size = df_ocupados_publico.groupby(['CH04', var]).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
n_pob = df_ocupados_publico.groupby(['CH04', var])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']
size.index = size.index.map(dict_cod_provincia)
empleo_publico = pd.concat([df_empleo_publico, size], axis=1)
empleo_publico = empleo_publico.sort_index()
empleo_publico.to_csv(pathdata + 'data_output/indicadores_auxiliares/01_empleo_publico.csv')

In [12]:
## Distribución de población ocupada, según intensidad laboral

df_ocupados_intensi = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'CH04','INTENSI']]
df_tasa = df_ocupados_intensi.groupby(['CH04',var,'INTENSI'])[pondera].sum()/df_ocupados_intensi.groupby(['CH04',var])[pondera].sum()
df_intensidad_laboral = df_tasa.unstack(level=0)*100
df_intensidad_laboral = df_intensidad_laboral.rename(columns={1:'Varon', 2:'Mujer'})
df_intensidad_laboral['Ratio'] = df_intensidad_laboral['Mujer']/df_intensidad_laboral['Varon']*100
new_index = df_intensidad_laboral.index.get_level_values('PROVINCIA').map(dict_cod_provincia)
df_intensidad_laboral.index = pd.MultiIndex.from_tuples(zip(new_index, df_intensidad_laboral.index.get_level_values('INTENSI')), names=['PROVINCIA', 'INTENSI'])

size = df_ocupados_intensi.groupby(['CH04', var, 'INTENSI']).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
size.index = pd.MultiIndex.from_tuples(zip(new_index, size.index.get_level_values('INTENSI')), names=['PROVINCIA', 'INTENSI'])

n_pob = df_ocupados_intensi.groupby(['CH04', var, 'INTENSI'])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
n_pob.index = pd.MultiIndex.from_tuples(zip(new_index, n_pob.index.get_level_values('INTENSI')), names=['PROVINCIA', 'INTENSI'])

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']
intensidad_laboral = pd.concat([df_intensidad_laboral, size], axis=1)
intensidad_laboral = intensidad_laboral.sort_index()

intensidad_laboral.to_csv(pathdata + 'data_output/indicadores_auxiliares/02_intensidad_laboral.csv')

In [13]:
## Distribución de población ocupada según máximo nivel educativo alcanzado

df_ocupados_educacion = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'CH04','CH12','CH13']]
df_ocupados_educacion['Educacion'] = pd.NaT

df_ocupados_educacion.loc[(df_ocupados_educacion['CH12'].isin([1,2,3,9])),'Educacion'] = 'Hasta secundario incompleto'
df_ocupados_educacion.loc[(df_ocupados_educacion['CH12'].isin([4,5])) & (df_ocupados_educacion['CH13']==2),'Educacion'] = 'Hasta secundario incompleto'
df_ocupados_educacion.loc[(df_ocupados_educacion['CH12'].isin([4,5])) & (df_ocupados_educacion['CH13']==1),'Educacion'] = 'Secundario completo'
df_ocupados_educacion.loc[df_ocupados_educacion['CH12'].isin([6,7,8]),'Educacion'] = 'Educación superior'
df_ocupados_educacion = df_ocupados_educacion[~df_ocupados_educacion['Educacion'].isna()]

df_tasa = df_ocupados_educacion.groupby(['CH04',var,'Educacion'])[pondera].sum()/df_ocupados_educacion.groupby(['CH04',var])[pondera].sum()

df_educacion = df_tasa.unstack(level=0)*100
df_educacion = df_educacion.rename(columns={1:'Varon', 2:'Mujer'})
df_educacion['Ratio'] = df_educacion['Mujer']/df_educacion['Varon']*100

new_index = df_educacion.index.get_level_values('PROVINCIA').map(dict_cod_provincia)
df_educacion.index = pd.MultiIndex.from_tuples(zip(new_index, df_educacion.index.get_level_values('Educacion')), names=['PROVINCIA', 'Educacion'])

size = df_ocupados_educacion.groupby(['CH04', var, 'Educacion']).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
size.index = pd.MultiIndex.from_tuples(zip(new_index, size.index.get_level_values('Educacion')), names=['PROVINCIA', 'Educacion'])

n_pob = df_ocupados_educacion.groupby(['CH04', var, 'Educacion'])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
n_pob.index = pd.MultiIndex.from_tuples(zip(new_index, n_pob.index.get_level_values('Educacion')), names=['PROVINCIA', 'Educacion'])

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']

educacion = pd.concat([df_educacion, size], axis=1)
educacion = educacion.sort_index()

educacion.to_csv(pathdata + 'data_output/indicadores_auxiliares/03_maximo_nivel_educativo.csv')

In [14]:
## Distribución de población ocupada / activa, según categorías ocupacionales (patrón, asalariado, cuentapropia, TFSR)

df_cat_ocup = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'CH04', 'CAT_OCUP']]
df_tasa = df_cat_ocup.groupby(['CH04',var,'CAT_OCUP'])[pondera].sum()/df_cat_ocup.groupby(['CH04',var])[pondera].sum()
df_poblacion_ocupada = df_tasa.unstack(level=0)*100
df_poblacion_ocupada = df_poblacion_ocupada.rename(columns={1:'Varon', 2:'Mujer'})
df_poblacion_ocupada['Ratio'] = df_poblacion_ocupada['Mujer']/df_poblacion_ocupada['Varon']*100
new_index = df_poblacion_ocupada.index.get_level_values('PROVINCIA').map(dict_cod_provincia)
df_poblacion_ocupada.index = pd.MultiIndex.from_tuples(zip(new_index, df_poblacion_ocupada.index.get_level_values('CAT_OCUP')), names=['PROVINCIA', 'CAT_OCUP'])

size = df_cat_ocup.groupby(['CH04', var, 'CAT_OCUP']).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
size.index = pd.MultiIndex.from_tuples(zip(new_index, size.index.get_level_values('CAT_OCUP')), names=['PROVINCIA', 'CAT_OCUP'])

n_pob = df_cat_ocup.groupby(['CH04', var, 'CAT_OCUP'])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
n_pob.index = pd.MultiIndex.from_tuples(zip(new_index, n_pob.index.get_level_values('CAT_OCUP')), names=['PROVINCIA', 'CAT_OCUP'])

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']
categorias_ocupacionales = pd.concat([df_poblacion_ocupada, size], axis=1)
categorias_ocupacionales = categorias_ocupacionales.sort_index()
categorias_ocupacionales.to_csv(pathdata + 'data_output/indicadores_auxiliares/04_categorias_ocupacionales.csv')

In [16]:
## Distribución de población asalariada, según descuento jubilatorio (con/sin) y aporte propio (con/sin)
%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb

resultados, tabla_pob_ocupada_asalariada = calcular_tabla_poblacion_ocupada_asalariada(df_people_pool, tipo='Aglomerado', base='Individual', pool=True)
resultados.to_csv(pathdata + 'data_output/indicadores_auxiliares/05_poblacion_asalariada.csv')

In [83]:
## Tasa de pluriempleo nacional

df_pluriempleo = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'CH04', 'PP03C', 'PP03D', 'PP04B1', 'PP04B2']]
cond1 = df_pluriempleo['PP03C'].isin([0, 1])
cond2 = df_pluriempleo['PP03C'] == 2
cond3 = (df_pluriempleo['PP03D'] > 1) & (df_pluriempleo['PP04B1'] == 1) & (df_pluriempleo['PP04B2'] > 2)

qv_tot = df_pluriempleo.groupby('CH04')[pondera].sum()

resultados = pd.DataFrame(index=['PP03C==0 | PP03C==1', 'PP03C==2', 'PP03D>1 & PP04B1==1 & PP04B2>2', 'Total Col'], columns=[ 'Varones', 'Mujeres', 'Total V + M', '% Varones', '% Mujeres', 'Ratio', ])
resultados.loc['PP03C==0 | PP03C==1',['Varones', 'Mujeres']] = df_pluriempleo[cond1].groupby('CH04')[pondera].sum().sort_index().values
resultados.loc['PP03C==2',['Varones', 'Mujeres']] = df_pluriempleo[cond2].groupby('CH04')[pondera].sum().sort_index().values
resultados.loc['PP03D>1 & PP04B1==1 & PP04B2>2',['Varones', 'Mujeres']] = df_pluriempleo[cond3].groupby('CH04')[pondera].sum().sort_index().values
resultados['Total V + M'] = resultados['Varones'] + resultados['Mujeres']

resultados.loc['Total Col'] = resultados.sum(axis=0)

resultados.loc[:, '% Varones'] = resultados['Varones'] / resultados.loc['Total Col', 'Varones'] * 100
resultados.loc[:, '% Mujeres'] = resultados['Mujeres'] / resultados.loc['Total Col', 'Mujeres'] * 100
resultados.loc[:, 'Ratio'] = resultados['% Mujeres'] / resultados['% Varones']
resultados.to_csv(pathdata + 'data_output/indicadores_auxiliares/06_pluriempleo_nacional.csv')    

In [82]:
## Absolutos pluriempleo nacional

df1 = df_pluriempleo[cond1].groupby([var, 'CH04'])[pondera].sum().unstack(level=1).rename(columns={1: 'Varones PP03C==0 | PP03C==1', 2: 'Mujeres PP03C==0 | PP03C==1'})
df1_size = df_pluriempleo[cond1].groupby([var, 'CH04'])[pondera].size().unstack(level=1).rename(columns={1: 'Nv', 2: 'Nm'})

df2 = df_pluriempleo[cond2].groupby([var, 'CH04'])[pondera].sum().unstack(level=1).rename(columns={1: 'Varones PP03C==2', 2: 'Mujeres PP03C==2'})
df2_size = df_pluriempleo[cond2].groupby([var, 'CH04'])[pondera].size().unstack(level=1).rename(columns={1: 'Nv', 2: 'Nm'})

df3 = df_pluriempleo[cond3].groupby([var, 'CH04'])[pondera].sum().unstack(level=1).rename(columns={1: 'Varones PP03D>1 & PP04B1==1 & PP04B2>2', 2: 'Mujeres PP03D>1 & PP04B1==1 & PP04B2>2'})
df3_size = df_pluriempleo[cond3].groupby([var, 'CH04'])[pondera].size().unstack(level=1).rename(columns={1: 'Nv', 2: 'Nm'})

pluriempleo_absolutos = pd.concat([df1, df1_size, df2, df2_size, df3, df3_size], axis=1)
pluriempleo_absolutos.index = pluriempleo_absolutos.index.map(dict_cod_provincia)
pluriempleo_absolutos.to_csv(pathdata + 'data_output/indicadores_auxiliares/07_pluriempleo_provincial_absolutos.csv')    

In [97]:
## Ingresos laborales secundarios

pondera = 'PONDII' 
df_ocupados = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'TOT_P12_new', 'CH04']]
df_ocupados = df_ocupados[(df_ocupados['TOT_P12_new']>0) & (df_ocupados['TOT_P12_new']!=-9)]
df_ocupados['Multiplication'] = df_ocupados['TOT_P12_new'] * df_ocupados[pondera]
numerador = df_ocupados.groupby(['CH04', var])['Multiplication'].sum().unstack(level=0)
denominador = df_ocupados.groupby(['CH04', var])[pondera].sum().unstack(level=0)

size = df_ocupados.groupby(['CH04', var]).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
n_pob = df_ocupados.groupby(['CH04', var])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']

tasa = numerador.div(denominador, fill_value=np.nan).rename(columns={1: 'Varon', 2: 'Mujer'}) 
tasa.loc[:, 'Ratio'] = tasa['Mujer']/tasa['Varon']*100
ingresos_laborales_secundarios = pd.concat([tasa, size], axis=1)
ingresos_laborales_secundarios.index = ingresos_laborales_secundarios.index.map(dict_cod_provincia)
ingresos_laborales_secundarios = ingresos_laborales_secundarios.sort_index()
ingresos_laborales_secundarios.to_csv(pathdata + 'data_output/indicadores_auxiliares/08_ingresos_laborales_secundarios.csv')

In [105]:
## Ingreso secundario
%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones_insercion_laboral.ipynb

tasa, size, ratio, error = horas_promedio_remuneradas_secundarias(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
horas_trabajadas_secundarias = pd.concat([tasa, size, ratio], axis=1)
horas_trabajadas_secundarias.index = horas_trabajadas_secundarias.index.map(dict_cod_provincia)
horas_trabajadas_secundarias = horas_trabajadas_secundarias.sort_index()
horas_trabajadas_secundarias.to_csv(pathdata + 'data_output/indicadores_auxiliares/09_horas_remuneradas_secundarias.csv')    

### ENUT

In [110]:
## Calculo coeficientes para imputar a PBA
df_temp = df_people_pool[df_people_pool['CH06']>14]
df_pob = df_temp.groupby(['AGLOMERADO']).sum(['PONDERA_new'])
df_pob.index = df_pob.index.map(dict_cod_aglomerado)
df_pob['PROVINCIA'] = df_pob.index.map(map_aglomerado_provincia)
pob_pba = df_pob[df_pob['PROVINCIA']=='Buenos Aires']['PONDERA_new'].sum()
gba = df_pob[df_pob['PROVINCIA']=='Buenos Aires'].loc['Partidos del GBA', 'PONDERA_new']
resto_pba = df_pob[df_pob['PROVINCIA']=='Buenos Aires'].loc[['Gran La Plata', 'Bahía Blanca - Cerri', 'Mar del Plata', 'Resto Buenos Aires'], 'PONDERA_new'].sum()
coef_gba = gba/pob_pba
coef_resto_pba = resto_pba/pob_pba


In [116]:
# Calculo TNR
pondera = 'WPER'
df_temp_enut = df_enut[df_enut['EDAD_SEL']>=14]
df_selected = df_temp_enut[(~df_temp_enut['TCS_GRANGRUPO_TNR'].isna())][['TCS_GRANGRUPO_TNR', 'SEXO_SEL', 'REGION', pondera]]
df_selected['Multiplication'] = df_selected['TCS_GRANGRUPO_TNR'] * df_selected[pondera]

numerador = df_selected.groupby(['SEXO_SEL', 'REGION'])['Multiplication'].sum().unstack(level=0)
denominador = df_selected.groupby(['SEXO_SEL', 'REGION'])[pondera].sum().unstack(level=0)
tasa = numerador.div(denominador, fill_value=np.nan)    
tasa = tasa.rename(columns= {2:'Varon', 1:'Mujer'})

size = df_temp_enut[(~df_temp_enut['TCS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL']).size().unstack(level=1)
size = size.rename(columns= {2:'N_v', 1:'N_m'})

n_pob = df_temp_enut[(~df_temp_enut['TCS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL'])[pondera].sum().unstack(level=1)
n_pob = n_pob.rename(columns= {2:'N_pob_v', 1:'N_pob_m'})

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m'] 

ratio = tasa['Varon'] / tasa['Mujer']

error = pd.DataFrame(index=size.index, columns=['LI', 'LS', 'ME', 'ER'])

ratio_minutos_promedio_no_pago_tcs = pd.concat([tasa, size, ratio*100, error], axis=1)
ratio_minutos_promedio_no_pago_tcs.rename(columns={0: 'TCS TNR'}, inplace=True)

ratio_minutos_promedio_no_pago_tcs.index = ratio_minutos_promedio_no_pago_tcs.index.map(dict_regiones_ENUT)

# Imputo de regiones a provincias
map_provincia_region['Ciudad Autónoma de Buenos Aires'] = 'Gran Buenos Aires'
provincias = list(map_provincia_region.keys())
regiones = [map_provincia_region[provincia] for provincia in provincias]

ratio_minutos_promedio_no_pago_tcs_provincial = pd.DataFrame(index=provincias, columns=['Mujer', 'Varon'])
for provincia in provincias:
    region = map_provincia_region[provincia]
    ratio_minutos_promedio_no_pago_tcs_provincial.loc[provincia,:] = ratio_minutos_promedio_no_pago_tcs.loc[region,['Mujer', 'Varon']]

ratio_minutos_promedio_no_pago_tcs_provincial.index.name = 'Provincia'

## Corrijo el caso de PBA
ratio_minutos_promedio_no_pago_tcs_provincial.loc['Buenos Aires', ['Mujer', 'Varon']] = ratio_minutos_promedio_no_pago_tcs.loc['Gran Buenos Aires', ['Mujer', 'Varon']]*coef_gba + \
                                        ratio_minutos_promedio_no_pago_tcs.loc['Pampeana', ['Mujer', 'Varon']]*coef_resto_pba

# Calculo ratio
ratio_minutos_promedio_no_pago_tcs_provincial['TCS TNR'] = ratio_minutos_promedio_no_pago_tcs_provincial['Varon'] / ratio_minutos_promedio_no_pago_tcs_provincial['Mujer']*100
ratio_minutos_promedio_no_pago_tcs_provincial = ratio_minutos_promedio_no_pago_tcs_provincial.rename(index={'Ciudad Autónoma de Buenos Aires':'CABA', 'Tierra del Fuego, Antártida e Islas del Atlántico Sur':'TdF'})

# Exporto csv
ratio_minutos_promedio_no_pago_tcs = ratio_minutos_promedio_no_pago_tcs.sort_index()
ratio_minutos_promedio_no_pago_tcs_provincial = ratio_minutos_promedio_no_pago_tcs_provincial.sort_index()

ratio_minutos_promedio_no_pago_tcs_provincial.to_csv(pathdata + 'data_output/indicadores_auxiliares/10_tiempo_no_remunerado_TCS.csv')
ratio_minutos_promedio_no_pago_tcs.to_csv(pathdata + 'data_output/indicadores_auxiliares/10_tiempo_no_remunerado_regional_TCS.csv')

In [117]:
df_selected = df_temp_enut[(~df_temp_enut['TP_GRANGRUPO_TNR'].isna())][['TP_GRANGRUPO_TNR', 'SEXO_SEL', 'REGION', pondera]]


df_selected

,TP_GRANGRUPO_TNR,SEXO_SEL,REGION,WPER
0,1,2,1,1580
1,1,1,1,2175
2,1,1,1,5078
3,1,2,1,2182
4,1,2,1,10202
...,...,...,...,...
14345,0,2,6,536
14346,1,1,6,423
14347,1,2,6,535
14348,1,1,6,423


In [27]:
def weighted_median(values, weights):
    sorted_indices = np.argsort(values)
    sorted_values = values[sorted_indices]
    sorted_weights = weights[sorted_indices]
    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = cumulative_weights[-1]
    median_index = np.searchsorted(cumulative_weights, total_weight / 2.0)
    return sorted_values[median_index]

def weighted_median_group(x):
    return weighted_median(x['P21_new'].values, x[pondera].values)


df_mediana = df_ocupados.groupby(['CH04', var]).apply(weighted_median_group)
tasa = df_mediana.unstack(level=0)
tasa.columns = ['Varon', 'Mujer']

In [4]:
# Calculo para Juanma

df_people_3T2021 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_21.txt', low_memory=False, index_col=0, delimiter=';')
df_temp = df_people_3T2021.query('CH06 >= 14')

df_estado = df_temp[(df_temp['ESTADO']==1)]          # personas ocupadas
df_ocupados = df_temp[(df_temp['P21']!=-9) & (df_temp['ESTADO']==1) & (df_temp['CAT_OCUP']==3)][['PONDIIO', 'P21', 'CH04']]
df_ocupados = df_ocupados[df_ocupados['P21']>0]

df_ocupados['Multiplication'] = df_ocupados['P21'] * df_ocupados['PONDIIO']
numerador = df_ocupados.groupby(['CH04'])['Multiplication'].sum()
denominador = df_ocupados.groupby(['CH04'])['PONDIIO'].sum()
tasa = numerador.div(denominador, fill_value=np.nan)    
tasa = tasa.rename(index= {1:'Varon', 2:'Mujer'})
tasa.loc['Ratio'] = tasa['Mujer']/tasa['Varon']*100
tasa